In [1]:
import os
import numpy as np
import pandas as pd
import pickle
import quandl
from datetime import datetime

In [2]:
import plotly.offline as py
import plotly.graph_objs as go
import plotly.figure_factory as ff
py.init_notebook_mode(connected=True)

In [3]:
def get_quandl_data(quandl_id):
    ''' Quandlの時系列データのキャッシュを取得'''
    cache_path = '{}.pkl'.format(quandl_id).replace('/','-')
    try:
        f = open(cache_path, 'rb')
        df = pickle.load(f)   
        print('Loaded {} from cache'.format(quandl_id))
    except (OSError, IOError) as e:
        print('Downloading {} from Quandl'.format(quandl_id))
        df = quandl.get(quandl_id, returns="pandas")
        df.to_pickle(cache_path)
        print('Cached {} at {}'.format(quandl_id, cache_path))
    return df

In [4]:
btc_usd_price_kraken = get_quandl_data('BCHARTS/KRAKENUSD')

Cached BCHARTS/KRAKENUSD at BCHARTS-KRAKENUSD.pkl


In [5]:
btc_usd_price_kraken.head()

,Open,High,Low,Close,Volume (BTC),Volume (Currency),Weighted Price
Date,,,,,,,
2014-01-07,874.67040,892.06753,810.00000,810.00000,15.622378,13151.472844,841.835522
2014-01-08,810.00000,899.84281,788.00000,824.98287,19.182756,16097.329584,839.156269
2014-01-09,825.56345,870.00000,807.42084,841.86934,8.158335,6784.249982,831.572913
2014-01-10,839.99000,857.34056,817.00000,857.33056,8.024510,6780.220188,844.938794
2014-01-11,858.20000,918.05471,857.16554,899.84105,18.748285,16698.566929,890.671709


In [6]:
# Chart the BTC pricing data
btc_trace = go.Scatter(x=btc_usd_price_kraken.index, y=btc_usd_price_kraken['Weighted Price'])
py.iplot([btc_trace])

In [7]:
# 他の交換所のレートを取得する
exchanges = ['COINBASE', 'BITSTAMP', 'ITBIT']
exchange_data = {}
exchange_data['KRAKEN'] = btc_usd_price_kraken

for exchange in exchanges:
    exchange_code = 'BCHARTS/{}USD'.format(exchange)
    btc_exchange_df = get_quandl_data(exchange_code)
    exchange_data[exchange]=btc_exchange_df

def merge_dfs_on_column(dataframes, labels, col):
    '''データフレームへ新しいカラムを追加する'''
    series_dict = {}
    for index in range(len(dataframes)):
        series_dict[labels[index]] = dataframes[index][col]

    return pd.DataFrame(series_dict)

btc_usd_datasets = merge_dfs_on_column(list(exchange_data.values()), list(exchange_data.keys()), 'Weighted Price')

# 各交換所のビットコインレートの平均をとる。
btc_usd_datasets['avg_btc_price_usd']=btc_usd_datasets.mean(axis=1)

Cached BCHARTS/COINBASEUSD at BCHARTS-COINBASEUSD.pkl
Cached BCHARTS/BITSTAMPUSD at BCHARTS-BITSTAMPUSD.pkl
Cached BCHARTS/ITBITUSD at BCHARTS-ITBITUSD.pkl


In [8]:
def get_json_data(json_url, cache_path):
    '''Download and cache JSON data, return as a dataframe.'''
    try:        
        f = open(cache_path, 'rb')
        df = pickle.load(f)   
        print('Loaded {} from cache'.format(json_url))
    except (OSError, IOError) as e:
        print('Downloading {}'.format(json_url))
        df = pd.read_json(json_url)
        df.to_pickle(cache_path)
        print('Cached {} at {}'.format(json_url, cache_path))
    return df

In [9]:
base_polo_url = 'https://poloniex.com/public?command=returnChartData&currencyPair={}&start={}&end={}&period={}'
start_date = datetime.strptime('2015-01-01', '%Y-%m-%d') # get data from the start of 2015
end_date = datetime.now() # up until today
pediod = 86400 # pull daily data (86,400 seconds per day)

def get_crypto_data(poloniex_pair):
    '''Retrieve cryptocurrency data from poloniex'''
    json_url = base_polo_url.format(poloniex_pair, start_date.timestamp(), end_date.timestamp(), pediod)
    data_df = get_json_data(json_url, poloniex_pair)
    data_df = data_df.set_index('date')
    return data_df

In [10]:
altcoins = ['ETH','LTC','XRP','ETC','STR','DASH','SC','XMR','XEM']

def get_altcoin():
    altcoin_data = {}
    for altcoin in altcoins:
        coinpair = 'BTC_{}'.format(altcoin)
        crypto_price_df = get_crypto_data(coinpair)
        altcoin_data[altcoin] = crypto_price_df
    return(altcoin_data)

In [14]:
altcoin_data = get_altcoin()

Loaded https://poloniex.com/public?command=returnChartData&currencyPair=BTC_ETH&start=1420038000.0&end=1562642882.142678&period=86400 from cache
Loaded https://poloniex.com/public?command=returnChartData&currencyPair=BTC_LTC&start=1420038000.0&end=1562642882.142678&period=86400 from cache
Loaded https://poloniex.com/public?command=returnChartData&currencyPair=BTC_XRP&start=1420038000.0&end=1562642882.142678&period=86400 from cache
Loaded https://poloniex.com/public?command=returnChartData&currencyPair=BTC_ETC&start=1420038000.0&end=1562642882.142678&period=86400 from cache
Loaded https://poloniex.com/public?command=returnChartData&currencyPair=BTC_STR&start=1420038000.0&end=1562642882.142678&period=86400 from cache
Loaded https://poloniex.com/public?command=returnChartData&currencyPair=BTC_DASH&start=1420038000.0&end=1562642882.142678&period=86400 from cache
Loaded https://poloniex.com/public?command=returnChartData&currencyPair=BTC_SC&start=1420038000.0&end=1562642882.142678&period=86

In [15]:
# それぞれのアルトコインのデータフレームへ新しいカラムとしてUSDの価格を計算する
for altcoin in altcoin_data.keys():
    altcoin_data[altcoin]['price_usd'] =  altcoin_data[altcoin]['weightedAverage'] * btc_usd_datasets['avg_btc_price_usd']

# 単一のデータフレームへ統合する。
combined_df = merge_dfs_on_column(list(altcoin_data.values()), list(altcoin_data.keys()), 'price_usd')

# データフレームへBTC価格を追加
combined_df['BTC'] = btc_usd_datasets['avg_btc_price_usd']

In [16]:
combined_df_2018 = combined_df[combined_df.index.year == 2018]
combined_df_2018.pct_change().corr(method='pearson')

,ETH,LTC,XRP,ETC,STR,DASH,SC,XMR,XEM,BTC
ETH,1.000000,0.931315,0.867417,0.910585,0.825230,0.918435,0.838221,0.927007,0.855133,0.936102
LTC,0.931315,1.000000,0.859827,0.883219,0.826523,0.918908,0.851964,0.921770,0.835825,0.945430
XRP,0.867417,0.859827,1.000000,0.828239,0.883937,0.849092,0.826077,0.855065,0.860746,0.860682
ETC,0.910585,0.883219,0.828239,1.000000,0.791441,0.875711,0.792197,0.873410,0.809772,0.888150
STR,0.825230,0.826523,0.883937,0.791441,1.000000,0.828938,0.820343,0.836810,0.855605,0.835768
DASH,0.918435,0.918908,0.849092,0.875711,0.828938,1.000000,0.840488,0.923260,0.842661,0.916703
SC,0.838221,0.851964,0.826077,0.792197,0.820343,0.840488,1.000000,0.843004,0.831757,0.846745
XMR,0.927007,0.921770,0.855065,0.873410,0.836810,0.923260,0.843004,1.000000,0.852936,0.939477
XEM,0.855133,0.835825,0.860746,0.809772,0.855605,0.842661,0.831757,0.852936,1.000000,0.849935
BTC,0.936102,0.945430,0.860682,0.888150,0.835768,0.916703,0.846745,0.939477,0.849935,1.000000


In [17]:
altcoins = ['ETH','LTC','XRP','ETC','STR','DASH','SC','XMR','XEM', 'BTC']

traced = go.Splom(dimensions=[dict(label=altcoin, values=combined_df_2018[altcoin]) for altcoin in altcoins],
                  marker=dict(#color=color_vals,
                              size=5,
                              #colorscale=pl_colorscaled,
                              line=dict(width=0.5,
                                        color='rgb(230,230,230)') ),
                  #text=textd,
                  diagonal=dict(visible=True))
fig = dict(data=[traced]) #, layout=layout)
py.iplot(fig, filename='large')